### Convert All Rasters in Folder to Shapefile ###

In [ ]:
# Convert using GRASS GIS

import os
from gasp.to.shp  import rst_to_polyg
from gasp.oss     import list_files
from gasp.cpu.grs import run_grass
from gasp.oss import get_filename

GRASS_WORKSPACE = r'D:\indev\xyz'
GRASS_LOCATION  = 'conv_viena'
RST_FOLDER = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\viena'
OUT_FOLDER = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\viena'
SRS = 3857
RST_TEMPLATE = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\viena\v14_viena_2x2.tif'

# List Raster Files
rstFiles = list_files(RST_FOLDER, file_format='tif')

# Start GRASS GIS Session
grsbase = run_grass(
    GRASS_WORKSPACE, location=GRASS_LOCATION, srs=SRS)

import grass.script.setup as gsetup
gsetup.init(grsbase, GRASS_WORKSPACE, GRASS_LOCATION, 'PERMANENT')

from gasp.to.rst.grs import rst_to_grs
from gasp.cpu.grs.conf import rst_to_region
from gasp.to.shp.grs import grs_to_shp

# Set region
tmpRst = rst_to_grs(RST_TEMPLATE, get_filename(RST_TEMPLATE), as_cmd=True)
rst_to_region(tmpRst)

for rst in rstFiles:
    # Add data to GRASS GIS
    grsRst = rst_to_grs(rst, get_filename(rst), as_cmd=True)
    
    # Convert to Polygon
    shpGrs = rst_to_polyg(
        grsRst, grsRst + "_shp",
        rstColumn="value", gisApi='grasscmd'
    )
    
    # Export to File
    grs_to_shp(shpGrs, os.path.join(OUT_FOLDER, grsRst + '.shp'), 'area')

### All Nc in Folders to tiff ###

In [ ]:
from gasp.to.rst.gdl import folder_nc_to_tif

folder_nc_to_tif(
    r'D:\gis\UNDERSee\sentinel_3', r'D:\gis\UNDERSee\sentinel_3'
)

### PostgreSQL Table to Feature Class with OGR Driver ###

In [ ]:
from gasp.to.shp import psql_to_shp

conParam = {
    "HOST" : "localhost", "USER" : "postgres", "PORT" : "5432",
    "PASSWORD" : "admin", "DATABASE" : "v12_londres"
}

TABLE = "near_roads"
OUTPUT = r'D:\indev\xyz\roads_london.shp'
SRS_CODE = 3857
GEOMETRY_COLUMN = "geometry"

psql_to_shp(
    conParam, TABLE, OUTPUT,
    epsg=SRS_CODE, geom_col=GEOMETRY_COLUMN,
    tableIsQuery=None, api='pgsql2shp'
)

### Raster to PostgreSQL Table ###

Este método tem um problema específico: num GeoTiff, os dados estão comprimidos, assim, um ficheiro 
com 6GB pode ter poucos MB porque está compactado; quando convertido para tabela PSQL, esses dados, em princípio, ocuparão 6GB.
Há forma de compactar os dados dentro do PostgreSQL?

In [ ]:
import os
from gasp.oss     import get_filename
from gasp.to.psql import rst_to_psql

IN_RASTER = r'D:\gis\EXT_CGI\ref_grids\grid_pt10x10.tif'
CON_PARAM = {
    'HOST': 'localhost', 'PORT': '5432',
    'PASSWORD': 'admin', 'USER': 'postgres',
    'DATABASE': 'expvgi_sdi'
}
EPSG = 3763

rst_to_psql(IN_RASTER, EPSG, lnk=CON_PARAM, sql_script=os.path.join(
    os.path.dirname(IN_RASTER), get_filename(IN_RASTER) + '.sql'
))

### Folder with TXT Files to PostgreSQL Database ###

Create a new database and add each TXT in Folder as PSQL Table. 

In [ ]:
from gasp.topsql import txts_to_newdb

CON_PARAM = {
    "HOST"     : "localhost", "PORT" : "5432", "USER" : "postgres",
    "PASSWORD" : "admin", "TEMPLATE" : "postgis_template"
}

TXT_FOLDER = r'D:\gis\TRENMO_WEBAPP\GTFS'

NEW_DB = "trenmogtfs"

DELIMITER = ","

txts_to_newdb(CON_PARAM, TXT_FOLDER, NEW_DB, DELIMITER, rewrite=True)

### All tables in PostgreSQL Database to TXT File ###

Each table in one database is converted to a single TXT File.

In [ ]:
from gasp.to.txt import psqldb_to_txt

SPLITTER   = '\t'
OUT_FOLDER = '/home/jasp/xpto/expvgi-webapp/old_models'
conParam   = {
    "HOST" : "localhost", "USER" : "postgres", "PORT" : "5432",
    "PASSWORD" : "admin", "DATABASE" : "expvgi_db"
}

psqldb_to_txt(SPLITTER, OUT_FOLDER, conParam)

### DB Table to XLS File ###

In [1]:
from gasp.to.xls import dbtbl_to_xls

QS = (
    "SELECT id, key, value, buffer_dist FROM osm_features "
    "INNER JOIN osm_ua ON osm_features.id = osm_ua.osm_id "
    "WHERE rule_id = 2"
)

XLS = r'C:\indev\osm_highway.xlsx'
SHEETS = 'highway'

CON_DB = r'C:\xpto\gasp\gasp\osm2lulc\osmtolulc.sqlite'

DB_API = 'sqlite'

dbtbl_to_xls(QS, XLS, SHEETS, CON_DB, dbapi=DB_API)

'C:\\indev\\osm_highway.xlsx'